In [5]:
from numpy import sqrt
def euclidienne(x1,y1, x2,y2):
    d = sqrt((x1-x2)**2 + (y1-y2)**2)
    return d
def centre(x,y,w,h):
    cx = x + (w-x)//2
    cy = y + (h-y)//2
    return (cx, cy)

In [6]:
import cv2
import numpy as np

In [7]:
#Chargez notre détecteur d'objets YOLO
net = cv2.dnn.readNetFromDarknet('Yolo4/yolov4.cfg', 'Yolo4/yolov4.weights')
ln = net.getLayerNames()
ln = [ln[i - 1] for i in net.getUnconnectedOutLayers()]

DEFAULT_CONFIANCE = 0.5 # la probabilité
THRESHOLD = 0.4 #un seuil qui va permettre d'éliminer la mauvaise détection
#Charger les étiquettes de classe COCO sur lesquelles notre modèle YOLO a été formé
with open('Yolo4/coco.names', 'r') as f:
    LABELS = f.read().splitlines()

#Initialiser le flux vidéo, pointeur vers le fichier vidéo de sortie
cap = cv2.VideoCapture("../vedios/people.mp4")
N = int(input("Saisir la distance: "))
while True:
    _,image=cap.read()
    # les coordonnées d'une image
    height, width, _ = image.shape 
    #préparer image pour le traitement
    #l'echelle :on va diviser chaque pixel par des 155
    #taille d'image => (416, 416)
    #swapRB=True    => converti des images qui sont passés par défaut en RGB
    #crop=False     => on ne veut pas couper notre image 
    blob = cv2.dnn.blobFromImage(image, 1/255, (416, 416), (0,0,0), swapRB=True, crop=False)
    net.setInput(blob)
    layerOutputs = net.forward(ln)
    
    #Initialiser nos listes de boîtes englobantes, de confiances et d'ID de classe détectés, respectivement
    boxes = [] #compter les différentes zones qui seront détectés son image
    confidences = [] # le niveau de confiance pour chacune des zones qui a été détecté 
    classIDs = []
    for output in layerOutputs:
        for detection in output:
            scores = detection[5:]
            classID = np.argmax(scores)
            confidence = scores[classID] # la probabilité avec laquelle cet objet à été détecté
            if confidence > DEFAULT_CONFIANCE:
                box = detection[0:4] * np.array([width, height, width, height])
                (centerX, centerY, W, H) = box.astype("int")
                x = int(centerX - (W / 2))
                y = int(centerY - (H / 2))
                boxes.append([x, y, int(W), int(H)])
                confidences.append(float(confidence))
                classIDs.append(classID)
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, DEFAULT_CONFIANCE, THRESHOLD)
    if len(indexes) > 0:
        for i in indexes.flatten():
            (x1,y1,w1,h1) = boxes[i]
            text = "{}: {:.4f}".format(LABELS[classIDs[i]], confidences[i])
            (cx1, cy1) = centre(x1,y1,w1,h1)
            for j in indexes.flatten():
                (x2,y2,w2,h2) = boxes[j]
                if x1!=x2 and y1!=y2:
                    (cx2, cy2) = centre(x2,y2,w2,h2)
                    distance = euclidienne(cx1, cy1, cx2, cy2)           
            if distance<N:
                cv2.rectangle(image,(x1,y1),(x1+w1,y1+h1),(0,0,255),2)
                cv2.putText(image, "Attention!", (x1, y1 - 15 ), cv2.FONT_HERSHEY_PLAIN, 2,(0,0,255),2)
            else:
                cv2.rectangle(image, (x1, y1), (x1 + w1, y1 + h1), (0,255,0), 2)
                cv2.putText(image, "personne", (x1, y1 - 15 ), cv2.FONT_HERSHEY_PLAIN, 2, (0,255,0), 2)
    cv2.imshow('Image', image)
    if cv2.waitKey(1)==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

Saisir la distance: 40


In [ ]:
del cv2
del numpy